In [ ]:
import findspark
findspark.init()
import os
import sys
file_path=os.getcwd()
sys.path.append(file_path)
import traceback
# from dforce.sf.forecast.sp.spark_bootstrap import predict_new_sku
# from dforce.sf.common.local.forcast_replenishment.mysql.common_operation import get_task
# from dforce.sf.common.local.common import setup_log
# from dforce.sf.forecast.sku_process.sku_category_group import *
# # import config as config
# from dforce.sf.common.local.common import save_model, load_model
# from dforce.sf.main_forecast import spark_forecast_method
# sys.path.append(config.sfroot)
from pyspark.sql import SparkSession
from pyhive import hive
import datetime
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, isnull
from pyspark.sql import Window
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

# from dforce.sf.main_replenish import spark_replenish_method
def spark_init():
    """
    初始化特征
    :return:
    """
    os.environ["PYSPARK_DRIVER_PYTHON"]="/data/ibs/anaconda3/bin/python"
    os.environ['PYSPARK_PYTHON']="/data/ibs/anaconda3/bin/python"
    spark=SparkSession.builder \
        .appName("gxc_test").master('yarn') \
        .config("spark.executor.instances","10")\
        .config("spark.executor.memory","6g")\
        .config("spark.executor.cores","4")\
        .config("spark.driver.memory", "4g")\
        .config("spark.driver.maxResultSize", "6g")\
        .config("spark.default.parallelism", "600")\
        .config("spark.network.timeout", "60s")\
        .config("spark.sql.adaptive.enabled", "true")\
        .config("spark.sql.adaptive.join.enabled", "true")\
        .config("spark.sql.adaptive.shuffle.targetPostShuffleInputSize", "128000000")\
        .config("spark.sql.hive.convertMetastoreParquet", "false")\
        .config("spark.dynamicAllocation.enabled", "true")\
        .config("spark.dynamicAllocation.minExecutors", "1")\
        .config("spark.shuffle.service.enabled", "true")\
        .enableHiveSupport().getOrCreate()
    spark.sql("set hive.exec.dynamic.partitions=true")
    spark.sql("set hive.exec.max.dynamic.partitions=2048")
    spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
    return spark
spark = spark_init()

In [ ]:

sparkdf_order = spark.table("ai_dm.ads_scm_goods_sales_log").\
select(['site_code','goods_code', 'quantity', 'paid_amount','dt']).filter("quantity>0") #~isnull('quantity')
sparkdf_order.show()

In [ ]:
dict_rename = {"site_code":"shop_id","goods_code":"goods_id","quantity":"qty","paid_amount":"sale_amt"}
def rename_columns(df, columns):

    if isinstance(columns, dict):
        for old_name, new_name in columns.items():

            df = df.withColumnRenamed(old_name, new_name)
        return df
    else:
        raise ValueError("'columns' should be a dict, like {'old_name_1':'new_name_1', 'old_name_2':'new_name_2'}")

sparkdf_order = rename_columns(sparkdf_order,dict_rename)
sparkdf_order.show()

In [ ]:
spark_price = sparkdf_order.groupby(['shop_id','goods_id','dt']).agg({'qty': 'sum','sale_amt':'sum'})\
.withColumnRenamed("sum(qty)","sum_qty")\
.withColumnRenamed("sum(sale_amt)","sum_sale_amt")
spark_price = spark_price.withColumn("price",spark_price['sum_sale_amt']/spark_price['sum_qty'])
spark_price.show()


In [ ]:
sparkdf_y = spark.table("ai_dm_dev.io_daily_y_tmp").select(['shop_id','goods_id','th_y','dt']).filter("y_type='c'")
sparkdf_y.show()

In [ ]:
df = spark_price.join(sparkdf_y,['shop_id','goods_id','dt'],'outer').filter(~isnull("th_y"))
df.show()

In [ ]:
df = df.withColumn("ts", F.unix_timestamp(F.to_timestamp(F.col('dt'), 'yyyyMMdd'),
                                                               "format='yyyy-MM-dd"))
df.show()

In [ ]:
df = df.fillna(0, subset=['sum_qty'])


In [ ]:

def days(i):
    return i * 86400
w = 10

windowOpt = Window.partitionBy(['goods_id','shop_id']).orderBy(F.col('ts')).rangeBetween(start=-days(w - 1),
                                                                                end=Window.currentRow)
df = df.withColumn("{0}_{1}_{2}".format('price', 'max', w),
                             F.max(F.col("{0}".format('price'))).over(windowOpt))
df.show()

In [ ]:
df = df.withColumn("{0}_{1}_{2}".format('sum_qty', 'sum', w),
                             F.sum(F.col("{0}".format('sum_qty'))).over(windowOpt))
df.show()

In [ ]:
df = df.withColumn("{0}_{1}_{2}".format('sum_sale_amt', 'sum', w),
                             F.sum(F.col("{0}".format('sum_sale_amt'))).over(windowOpt))
df.show()

In [ ]:
df = df.withColumn("price_mean",F.col("sum_sale_amt_sum_10")/F.col("sum_qty_sum_10"))
df.show()

In [ ]:
df = df.withColumn("price", (F.when((F.col("th_y")>0)&(F.col("sum_qty")==0),F.col('price_mean')).\
                              when((F.col("th_y")==0)&(F.col("sum_qty")==0),F.col('price_max_10')).otherwise(F.col("price"))))
df = df.withColumn("label_price", (F.when((F.col("th_y")>0)&(F.col("sum_qty")==0),F.lit('mean')).\
                              when((F.col("th_y")==0)&(F.col("sum_qty")==0),F.lit('max')).otherwise(F.lit("ori"))))
df.show()

In [ ]:
## 最后将真实销量用理论销量来代替
df = df.withColumn("label_qty",  F.when((F.col("sum_qty")<F.col("th_y"))|(F.col("sum_qty")==0),F.lit('adjust')).\
              otherwise(F.lit('ori')))
df = df.withColumn('sum_qty',
              F.when(F.col("sum_qty")<F.col("th_y"),F.col('th_y')).\
              otherwise(F.col('sum_qty')))

df.show()

In [ ]:
df = df.select(['shop_id','goods_id','dt','sum_qty','price','label_price','label_qty'])
df.show()

In [ ]:

def is_exist_table(spark,check_table):
    """
    判断表是否存在，存在不为空
    :param spark: spark
    :param check_table:检测的表
    :return: 不存在为空，存在不为空
    """
    result=""
    query_sql="""
        SHOW TABLES LIKE "{check_table}"
    """.format(check_table=check_table)
    try:
        datas=spark.sql(query_sql)
        tmp=datas.collect()
        if len(tmp)>0:
            result=tmp[0][1]
    except:
        pass
    return result

def write_to_hive(spark,data,partition,table_name,mode_type):
    """
    写入hive
    :param spark: spark
    :param data: 数据
    :param partition: partition
    :param table_name: 数据名
    :param mode_type: 方式：overwrite、append
    :return:
    """
    if is_exist_table(spark,table_name) != "":  # 表示表存在
        data.write.mode(mode_type).insertInto(table_name, True)  # 表示覆盖分区
    else:
        if partition!=[]:
            data.write.partitionBy(partition).mode(mode_type).saveAsTable(table_name)
        else:
            data.write.mode(mode_type).saveAsTable(table_name)

write_to_hive(spark,df,'dt','ai_dm_dev.price_elastic_mid_d','overwrite')